# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [25]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [26]:
# The path to our CSV file
file = "../output_data/weather_output.csv"

# Read our data into pandas
df = pd.read_csv(file)

df.rename(columns={'City Name': 'City', 'Lon': 'Lng'}, inplace=True)
df.head()

,City,City ID,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind speed,Country,Date
0,hobart,2163355,-42.8794,147.3294,64.99,72,75,12.66,AU,10/01/2021
1,ushuaia,3833367,-54.8000,-68.3000,44.60,87,75,4.61,AR,10/01/2021
2,effingham,4193042,32.3669,-81.3332,35.60,80,1,4.61,US,10/01/2021
3,dano,2361586,11.1500,-3.0667,84.51,15,0,4.90,BF,10/01/2021
4,mocuba,1024312,-16.8392,36.9856,100.45,41,100,1.72,MZ,10/01/2021


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [27]:
# Google developer API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)


In [28]:
# Store latitude and longitude in locations
locations = df[["Lat", "Lng"]]

# convert to float
rating = df["Humidity"].astype(float)

In [29]:
# Plot Heatmap
fig = gmaps.figure()
fig = gmaps.figure(layout={'padding': '3px','border': '1px solid black'}, )
fig = gmaps.figure(center=(-30.0, 130.0), zoom_level=3)
# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=10, opacity=0.5,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [32]:
# Create new DataFrame with my desired weather conditions

## Drop any rows will null values(not meeting all three conditions).

temp_df=df.loc[(df['Max Temp']>=70)&(df['Max Temp']<=91)]
wind_df=temp_df.loc[(temp_df['Wind speed']<=10)]
humid_df=wind_df.loc[(wind_df['Humidity']<70)]
ideal_df=humid_df.loc[(humid_df['Cloudiness']<30)]
ideal_df.head()


,City,City ID,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind speed,Country,Date
3,dano,2361586,11.1500,-3.0667,84.51,15,0,4.90,BF,10/01/2021
26,xai-xai,1024552,-25.0519,33.6442,87.82,49,0,5.82,MZ,10/01/2021
37,teahupoo,4033543,-17.8333,-149.2667,78.80,69,20,3.44,PF,10/01/2021
73,busselton,2075265,-33.6500,115.3333,72.00,61,0,5.99,AU,10/01/2021
98,palauig,1696188,15.4350,119.9045,79.59,63,0,4.94,PH,10/01/2021


In [34]:
## How does Map compare with only ideal locations - so does this look right?
# Store latitude and longitude in locations
locations = ideal_df[["Lat", "Lng"]]

# convert to float
rating = ideal_df["Humidity"].astype(float)
# Plot Heatmap
fig = gmaps.figure()
fig = gmaps.figure(layout={'padding': '3px','border': '1px solid black'}, )
fig = gmaps.figure(center=(-30.0, 130.0), zoom_level=3)
# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=10, opacity=0.5,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [40]:
hotel_df = ideal_df.reset_index(drop=True)
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,City ID,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind speed,Country,Date,Hotel Name
0,dano,2361586,11.1500,-3.0667,84.51,15,0,4.90,BF,10/01/2021,
1,xai-xai,1024552,-25.0519,33.6442,87.82,49,0,5.82,MZ,10/01/2021,
2,teahupoo,4033543,-17.8333,-149.2667,78.80,69,20,3.44,PF,10/01/2021,
3,busselton,2075265,-33.6500,115.3333,72.00,61,0,5.99,AU,10/01/2021,
4,palauig,1696188,15.4350,119.9045,79.59,63,0,4.94,PH,10/01/2021,


In [41]:
hotel=[]
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotels",
    "key": g_key
}

# read each line of the DF and use lat lon to find hotels
for index, row in hotel_df.iterrows():
    
    # get lat, lng 
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
##    hotel.append(hotel_name)
##    print(json.dumps(hotel_name, indent=4, sort_keys=True))

    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        pass
type(hotel_df)

pandas.core.frame.DataFrame

In [42]:
type(hotel_df)
## just checking the type again, as the next call 
## (which is supplied in this excerise) sometimes produces an error, that suggests it might be a list
## but not according to this

pandas.core.frame.DataFrame

In [43]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [46]:
# Add marker layer ontop of heat map

locations = ideal_df[["Lat", "Lng"]]

# convert to float
rating = ideal_df["Humidity"].astype(float)
# Plot Heatmap
fig = gmaps.figure()
fig = gmaps.figure(layout={'padding': '3px','border': '1px solid black'}, )
fig = gmaps.figure(center=(-30.0, 130.0), zoom_level=3)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=10, opacity=0.5,
                                 point_radius=1)

# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
)


# Add layer
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))